# What is a Kuzushiji-MNIST?

Hello everyone! Today, we are here with another very cool dataset **KMNIST**, short for **Kuzushiji-MNIST**.  This dataset was showcased at NeurIPS 2018 along with a [paper](https://arxiv.org/pdf/1812.01718.pdf).  The dataset is actually divided into three categories:<br>
* **Kuzushiji-MNIST** is a drop-in replacement for the MNIST dataset (28x28 grayscale, 70,000 images), provided in the original MNIST format as well as a NumPy format. Since MNIST restricts us to 10 classes, we chose one character to represent each of the 10 rows of Hiragana when creating Kuzushiji-MNIST.

* **Kuzushiji-49**, as the name suggests, has 49 classes (28x28 grayscale, 270,912 images), is a much larger, but imbalanced dataset containing 48 Hiragana characters and one Hiragana iteration mark.

* **Kuzushiji-Kanji** is an imbalanced dataset of total 3832 Kanji characters (64x64 grayscale, 140,426 images), ranging from 1,766 examples to only a single example per class.

In this kernel, we will focus on the **Kuzushiji-MNIST**  dataset. For many years, researchers have been using MNIST as one of the standard datasets for benchmarking their algorithms. No doubt that MNIST is a very good dataset but given the advancements that we have achieved in deep learning in the last 5 years, it is high time to replace MNIST with a more challenging dataset.  The above point is fair but why KMNIST? What makes it unique? Why is it important?

Actually,  cursive Kuzushiji (くずし字) was an integral part of Japanese culture in the pre-modernisation era. It has been used for more than 1000 years but with the reforms done by Japanese leaders in the Japanese education system in 1868 and the following years, Kuzushiji was no longer seen as an important part and as a result, it is no longer taught in the official school curriculum.  Now you may ask: "So what? Japanese lost the script but with modernisation, such things are expected. Why do you think it is important at all?"

Fair question! As a result of the changes that were made in the education system, most Japanese cannot read books over 150 years old, written in cursive Kuzushiji style. Also, According to the General Catalog of National Books, there have been over 1.7 million books written or published in Japan prior to 1867 along with a billion of unregistered books. **Almost every book, no matter what, always provides some sort of knowledge**.  So, in short, we have a lot of knowledge lying around in those books but we don't have many people who are experts in reading and extracting information from them. Because of lack of domain experts, the process of digitizing that knowledge is slow. 

This dataset can help non-experts to develop systems that can be implemented to extract the information from those millions and billions of books. Deep learning can certainly be very useful in this task. If you want more info about the topic, I suggest reading that paper I mentioned above. Let's dive into the dataset now and see how different is it from MNIST.  
 


![SophiaImitation](tarry2.png)

# Importing stuff


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

/home/deepkapha/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
num_classes = 10

# hyperparamters
batch_size = 96*4
epochs = 500
dropout_one = 0.5
dropout_two = 0.5
conv2d_one = 32
conv2d_two = conv2d_one*2

In [12]:
# img size
img_rows, img_cols = 28, 28

# kmnist data
x_train = np.load('./input/kmnist-train-imgs.npz')['arr_0']
x_test = np.load('./input/kmnist-test-imgs.npz')['arr_0']

y_train = np.load('./input/kmnist-train-labels.npz')['arr_0']
y_test = np.load('./input/kmnist-test-labels.npz')['arr_0']

# OG mnist data
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [13]:
# build our model

model = Sequential()
model.add(Conv2D(conv2d_one, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(conv2d_two, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout_one))

model.add(Conv2D(conv2d_two, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(conv2d_two*2, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout_one))

model.add(Flatten())
model.add(Dense(conv2d_two*4, activation='relu'))
model.add(Dropout(dropout_two))
model.add(Dense(num_classes, activation='softmax'))

In [14]:
# compile
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [15]:
# run the model

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/500
60000/60000 [==============================] - 4s 66us/step - loss: 0.7542 - acc: 0.7550 - val_loss: 0.5110 - val_acc: 0.8414
Epoch 2/500
60000/60000 [==============================] - 4s 62us/step - loss: 0.2547 - acc: 0.9196 - val_loss: 0.3502 - val_acc: 0.8935
Epoch 3/500
60000/60000 [==============================] - 4s 59us/step - loss: 0.1803 - acc: 0.9454 - val_loss: 0.2632 - val_acc: 0.9225
Epoch 4/500
60000/60000 [==============================] - 4s 62us/step - loss: 0.1368 - acc: 0.9576 - val_loss: 0.2251 - val_acc: 0.9362
Epoch 5/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.1145 - acc: 0.9642 - val_loss: 0.2144 - val_acc: 0.9399
Epoch 6/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0991 - acc: 0.9684 - val_loss: 0.1871 - val_acc: 0.9455
Epoch 7/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0856 - acc: 0.9734 - val_loss: 0.1701 - 

60000/60000 [==============================] - 4s 59us/step - loss: 0.0171 - acc: 0.9945 - val_loss: 0.1140 - val_acc: 0.9775
Epoch 60/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0174 - acc: 0.9944 - val_loss: 0.1075 - val_acc: 0.9778
Epoch 61/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0181 - acc: 0.9942 - val_loss: 0.1204 - val_acc: 0.9763
Epoch 62/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0174 - acc: 0.9943 - val_loss: 0.1054 - val_acc: 0.9779
Epoch 63/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0148 - acc: 0.9950 - val_loss: 0.1153 - val_acc: 0.9771
Epoch 64/500
60000/60000 [==============================] - 3s 58us/step - loss: 0.0158 - acc: 0.9953 - val_loss: 0.1138 - val_acc: 0.9775
Epoch 65/500
60000/60000 [==============================] - 4s 58us/step - loss: 0.0144 - acc: 0.9953 - val_loss: 0.1154 - val_acc: 0.9773
Epoch 66/500
60000/60000 [==============

Epoch 118/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0100 - acc: 0.9970 - val_loss: 0.1058 - val_acc: 0.9798
Epoch 119/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0113 - acc: 0.9964 - val_loss: 0.1168 - val_acc: 0.9787
Epoch 120/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0111 - acc: 0.9965 - val_loss: 0.1180 - val_acc: 0.9783
Epoch 121/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0089 - acc: 0.9968 - val_loss: 0.1243 - val_acc: 0.9770
Epoch 122/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0098 - acc: 0.9969 - val_loss: 0.1101 - val_acc: 0.9799
Epoch 123/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0113 - acc: 0.9966 - val_loss: 0.1126 - val_acc: 0.9801
Epoch 124/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0114 - acc: 0.9964 - val_loss: 0.1142 - val_acc: 0.9795
Epoch 125/500
60000/

60000/60000 [==============================] - 4s 60us/step - loss: 0.0098 - acc: 0.9969 - val_loss: 0.1222 - val_acc: 0.9785
Epoch 177/500
60000/60000 [==============================] - 4s 62us/step - loss: 0.0093 - acc: 0.9969 - val_loss: 0.1214 - val_acc: 0.9785
Epoch 178/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0092 - acc: 0.9969 - val_loss: 0.1196 - val_acc: 0.9793
Epoch 179/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0109 - acc: 0.9968 - val_loss: 0.1097 - val_acc: 0.9811
Epoch 180/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0089 - acc: 0.9972 - val_loss: 0.1159 - val_acc: 0.9803
Epoch 181/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0081 - acc: 0.9973 - val_loss: 0.1113 - val_acc: 0.9812
Epoch 182/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0078 - acc: 0.9975 - val_loss: 0.1104 - val_acc: 0.9816
Epoch 183/500
60000/60000 [=======

60000/60000 [==============================] - 4s 60us/step - loss: 0.0087 - acc: 0.9972 - val_loss: 0.1145 - val_acc: 0.9812
Epoch 235/500
60000/60000 [==============================] - 3s 57us/step - loss: 0.0082 - acc: 0.9974 - val_loss: 0.1151 - val_acc: 0.9789
Epoch 236/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0078 - acc: 0.9975 - val_loss: 0.1224 - val_acc: 0.9801
Epoch 237/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0098 - acc: 0.9970 - val_loss: 0.1157 - val_acc: 0.9801
Epoch 238/500
60000/60000 [==============================] - 3s 58us/step - loss: 0.0077 - acc: 0.9976 - val_loss: 0.1201 - val_acc: 0.9787
Epoch 239/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0086 - acc: 0.9973 - val_loss: 0.1241 - val_acc: 0.9799
Epoch 240/500
60000/60000 [==============================] - 3s 58us/step - loss: 0.0084 - acc: 0.9974 - val_loss: 0.1277 - val_acc: 0.9794
Epoch 241/500
60000/60000 [=======

60000/60000 [==============================] - 3s 55us/step - loss: 0.0081 - acc: 0.9977 - val_loss: 0.1255 - val_acc: 0.9824
Epoch 293/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0077 - acc: 0.9975 - val_loss: 0.1129 - val_acc: 0.9831
Epoch 294/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0093 - acc: 0.9970 - val_loss: 0.1204 - val_acc: 0.9806
Epoch 295/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0083 - acc: 0.9976 - val_loss: 0.1225 - val_acc: 0.9811
Epoch 296/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0073 - acc: 0.9978 - val_loss: 0.1156 - val_acc: 0.9822
Epoch 297/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0075 - acc: 0.9976 - val_loss: 0.1355 - val_acc: 0.9801
Epoch 298/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0083 - acc: 0.9975 - val_loss: 0.1036 - val_acc: 0.9825
Epoch 299/500
60000/60000 [=======

60000/60000 [==============================] - 4s 59us/step - loss: 0.0059 - acc: 0.9982 - val_loss: 0.1178 - val_acc: 0.9813
Epoch 351/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0076 - acc: 0.9978 - val_loss: 0.1114 - val_acc: 0.9809
Epoch 352/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0072 - acc: 0.9977 - val_loss: 0.1254 - val_acc: 0.9794
Epoch 353/500
60000/60000 [==============================] - 4s 59us/step - loss: 0.0078 - acc: 0.9979 - val_loss: 0.1208 - val_acc: 0.9823
Epoch 354/500
60000/60000 [==============================] - 3s 58us/step - loss: 0.0092 - acc: 0.9974 - val_loss: 0.1215 - val_acc: 0.9802
Epoch 355/500
60000/60000 [==============================] - 3s 58us/step - loss: 0.0075 - acc: 0.9978 - val_loss: 0.1246 - val_acc: 0.9806
Epoch 356/500
60000/60000 [==============================] - 3s 58us/step - loss: 0.0080 - acc: 0.9976 - val_loss: 0.1243 - val_acc: 0.9806
Epoch 357/500
60000/60000 [=======

60000/60000 [==============================] - 4s 59us/step - loss: 0.0071 - acc: 0.9978 - val_loss: 0.1115 - val_acc: 0.9821
Epoch 409/500
60000/60000 [==============================] - 4s 60us/step - loss: 0.0072 - acc: 0.9978 - val_loss: 0.1192 - val_acc: 0.9820
Epoch 410/500
60000/60000 [==============================] - 4s 58us/step - loss: 0.0090 - acc: 0.9975 - val_loss: 0.1255 - val_acc: 0.9805
Epoch 411/500
60000/60000 [==============================] - 4s 59us/step - loss: 0.0060 - acc: 0.9981 - val_loss: 0.1297 - val_acc: 0.9821
Epoch 412/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0082 - acc: 0.9978 - val_loss: 0.1230 - val_acc: 0.9810
Epoch 413/500
60000/60000 [==============================] - 4s 61us/step - loss: 0.0083 - acc: 0.9976 - val_loss: 0.1281 - val_acc: 0.9803
Epoch 414/500
60000/60000 [==============================] - 3s 57us/step - loss: 0.0074 - acc: 0.9978 - val_loss: 0.1288 - val_acc: 0.9798
Epoch 415/500
60000/60000 [=======

60000/60000 [==============================] - 3s 53us/step - loss: 0.0077 - acc: 0.9978 - val_loss: 0.1188 - val_acc: 0.9812
Epoch 467/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0076 - acc: 0.9977 - val_loss: 0.1271 - val_acc: 0.9811
Epoch 468/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0069 - acc: 0.9980 - val_loss: 0.1205 - val_acc: 0.9822
Epoch 469/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0078 - acc: 0.9977 - val_loss: 0.1220 - val_acc: 0.9823
Epoch 470/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0066 - acc: 0.9981 - val_loss: 0.1176 - val_acc: 0.9836
Epoch 471/500
60000/60000 [==============================] - 3s 53us/step - loss: 0.0069 - acc: 0.9980 - val_loss: 0.1301 - val_acc: 0.9820
Epoch 472/500
60000/60000 [==============================] - 3s 54us/step - loss: 0.0073 - acc: 0.9978 - val_loss: 0.1205 - val_acc: 0.9827
Epoch 473/500
60000/60000 [=======

In [16]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print('completed!')

Test loss: 0.12392997630835643
Test accuracy: 0.9823
completed!


** Really sweet going**

As reported in the paper, the results are fully reproducible. 
1. **Accuracy**: CNN is doing some cool wonders though $\color{red}{\text{~98.5%}}$ as compared to MNIST which folks (myself included) have pushed it to $\color{red}{\text{99.75%}}$ between the accuracy on MNIST and KMNIST. 
2. **Applicability**: This again proves the point that KMNIST is the perfect replacement for research and SOTA purposes as well. 
3. Next stop try with a bunch of stuff like Julia, Capsules Networks, AutoKeras, fast.ai and naturally our own activation functions and LayGO etc.
